In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [2]:
nQuizz=6
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/23-Quizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [4]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'ALINA': ['433401'],
 'Agathe': ['430440'],
 'Agathe / Liliana': ['430440', '430952'],
 'Agathe Martin': ['430440'],
 'Agathe martin': ['430440'],
 'Alex Helvert': ['380824'],
 'Alice': ['427886'],
 'Alice Gendre': ['428240'],
 'Alice LOOF': ['427886'],
 'Alice.Loof': ['427886'],
 'Alima Gassama': ['418366'],
 'Alissa Bessettes': ['429623'],
 'Alissa bessettes': ['429623'],
 'Alix DEBUR': ['427449'],
 u'Ali\xe9nor Riou': ['428930'],
 'Amandine B': ['429252'],
 'Amandine Berloin': ['429252'],
 'Amandine berloin': ['429252'],
 'Amelie Mitlehner': ['433485'],
 u'Am\xe9lie rochefort': ['432360'],
 u'Ana\xeblle DINA-SAME': ['432151'],
 'Auriault justin': ['430711'],
 'Axel Meunier': ['429855'],
 'BENTO ELENA': ['428999'],
 u'BENTO EL\xc9NA': ['428999'],
 'BENTO Elena': ['428999'],
 u'BORDELAIS cl\xe9a': ['429396'],
 u'BOUSSARD L\xe9a': ['428452'],
 'BOUSSARD MEUNIER': ['428452', '429855'],
 'Baptiste phelip': ['397583'],
 u'Barsacq Th\xe9o': ['403123'],
 'BarsacqTheo': ['403123'],
 'Barsac

In [5]:
len(alias)

331

In [6]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [7]:
# lQuizz, dfCM

# Listage des alias

In [8]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM

,id,nom,prénom,email,ine,PN,NP
0,415923,AKKUL,EMMANUEL,emmanuel.akkul@etu.u-bordeaux-montaigne.fr,1664938189,emmanuel akkul,akkul emmanuel
1,432822,AUBERT,EMMA,emma.aubert@etu.u-bordeaux-montaigne.fr,1664938189,emma aubert,aubert emma
2,430711,AURIAULT,JUSTIN,justin.auriault@etu.u-bordeaux-montaigne.fr,1664938189,justin auriault,auriault justin
3,431129,BARANGER,MANON,manon.baranger@etu.u-bordeaux-montaigne.fr,1664938189,manon baranger,baranger manon
4,403123,BARSACQ,THEO,theo.barsacq@etu.u-bordeaux-montaigne.fr,1664938189,theo barsacq,barsacq theo
5,433205,BEFORT,KRISTIN,kristin.befort@etu.u-bordeaux-montaigne.fr,1664938189,kristin befort,befort kristin
6,428999,BENTO,ELENA,elena.bento@etu.u-bordeaux-montaigne.fr,1664938189,elena bento,bento elena
7,429252,BERLOIN,AMANDINE,amandine.berloin@etu.u-bordeaux-montaigne.fr,1664938189,amandine berloin,berloin amandine
8,419149,BERTRAND,SASCHA,sascha.bertrand@etu.u-bordeaux-montaigne.fr,1664938189,sascha bertrand,bertrand sascha
9,429623,BESSETTES,ALISSA,alissa.bessettes@etu.u-bordeaux-montaigne.fr,1664938189,alissa bessettes,bessettes alissa


In [9]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

mathou
A
Lojban


In [10]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

mathou


Nom a chercher:
 


A


Nom a chercher:
 


Lojban


Nom a chercher:
 


In [11]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [12]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [13]:
pseudos

[u'mathou', u'A', u'Lojban']

In [14]:
# alias["Sofiane"]

In [15]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [16]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]



In [17]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [18]:
len(alias)

331

In [19]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'ALINA': ['433401'],
  'Agathe': ['430440'],
  'Agathe / Liliana': ['430440', '430952'],
  'Agathe Martin': ['430440'],
  'Agathe martin': ['430440'],
  'Alex Helvert': ['380824'],
  'Alice': ['427886'],
  'Alice Gendre': ['428240'],
  'Alice LOOF': ['427886'],
  'Alice.Loof': ['427886'],
  'Alima Gassama': ['418366'],
  'Alissa Bessettes': ['429623'],
  'Alissa bessettes': ['429623'],
  'Alix DEBUR': ['427449'],
  u'Ali\xe9nor Riou': ['428930'],
  'Amandine B': ['429252'],
  'Amandine Berloin': ['429252'],
  'Amandine berloin': ['429252'],
  'Amelie Mitlehner': ['433485'],
  u'Am\xe9lie rochefort': ['432360'],
  u'Ana\xeblle DINA-SAME': ['432151'],
  'Auriault justin': ['430711'],
  'Axel Meunier': ['429855'],
  'BENTO ELENA': ['428999'],
  u'BENTO EL\xc9NA': ['428999'],
  'BENTO Elena': ['428999'],
  u'BORDELAIS cl\xe9a': ['429396'],
  u'BOUSSARD L\xe9a': ['428452'],
  'BOUSSARD MEUNIER': ['428452', '429855'],
  'Baptiste phelip': ['397583'],
  u'Barsacq Th\xe9o': ['40

In [20]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [21]:
nbQ=6.0

In [22]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [23]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,Alice.Loof,5,130.04,[427886],18,427886,None,None
1,Maureen viard,4,106.07,[432229],17,432229,None,None
2,nell duplan,4,116.21,[431361],17,431361,None,None
3,ESPERON Astrid,4,136.71,[433135],17,433135,None,None
4,DETRIE margot,4,142.26,[428233],17,428233,None,None
5,Elise,3,57.29,[429273],15,429273,None,None
6,Susanna Deffner,3,79.78,[435053],15,435053,None,None
7,mathou,3,96.21,None,15,None,None,None
8,Sheryl BLANC,3,96.55,[427234],15,427234,None,None
9,Alex Helvert,3,97.14,[380824],15,380824,None,None


In [24]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [25]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
0,Alice.Loof,18,130.04,427886
1,Maureen viard,17,106.07,432229
2,nell duplan,17,116.21,431361
3,ESPERON Astrid,17,136.71,433135
4,DETRIE margot,17,142.26,428233
5,Elise,15,57.29,429273
6,Susanna Deffner,15,79.78,435053
8,Sheryl BLANC,15,96.55,427234
9,Alex Helvert,15,97.14,380824
10,Bordelais Cléa,15,101.01,429396


In [26]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [27]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"]<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,Alice.Loof,20,130.04,427886
1,Maureen viard,18,106.07,432229
2,nell duplan,17,116.21,431361
3,ESPERON Astrid,17,136.71,433135
4,DETRIE margot,17,142.26,428233
5,Elise,15,57.29,429273
6,Susanna Deffner,15,79.78,435053
8,Sheryl BLANC,15,96.55,427234
9,Alex Helvert,15,97.14,380824
10,Bordelais Cléa,15,101.01,429396


In [28]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")

In [29]:
dfNotes

,pseudo,note,temps
id,,,
427886,Alice.Loof,20,130.04
432229,Maureen viard,18,106.07
428233,DETRIE margot,17,142.26
433135,ESPERON Astrid,17,136.71
431361,nell duplan,17,116.21
380824,Alex Helvert,15,97.14
435053,Susanna Deffner,15,79.78
416084,Roukaya,15,136.31
425485,Shadrina Yelyzaveta,15,127.64
